Notebook to create the LaTeX tables from the data generated

In [1]:
data_file_location = "../data/output/framework1_hypkhyp.csv"

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import nltk
from nltk.corpus import wordnet as wn

from operations.composition import *
from operations.similarity_measures import *
from operations.logical_negation import *
from operations.worldly_context_creation import *
from operations.helpers import *

import pandas as pd
import matplotlib.pyplot as plt

import pickle
import csv
import random
import math


from matplotlib.lines import Line2D 
from matplotlib.ticker import MaxNLocator

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/benjaminrodatz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
file = []
df = None

with open(data_file_location, newline='') as csvfile:
        # with open('df_i4_all_idneg.csv', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            test = row
            file.append(row)

        df = pd.DataFrame(file[1:])
        df.columns = file[0]


In [4]:
# convert the rows after row 5 to numeric.
# It is assumed that these only contain numbers which can then be compared to the human rating.
# This cell prints all rows which have been found and will eventually be compared to the human rating.

df["MEANRATING"] = pd.to_numeric(df["MEANRATING"])
for col in df.columns[5:]:
    print(col)
    df[col] = pd.to_numeric(df[col])

rename = {}

for col in df.columns[5:]:      
    if "supp_ker" in col:
        rename[col] = "suppker" + col[8:]
df.rename(columns = rename, inplace = True)
print(rename)
    


support_diag_left_khyp
support_diag_left_swapkhyp
support_diag_left_kE
support_diag_left_swapkE
support_diag_left_kBA
support_diag_left_trace
support_diag_right_khyp
support_diag_right_swapkhyp
support_diag_right_kE
support_diag_right_swapkE
support_diag_right_kBA
support_diag_right_trace
support_phaser_left_khyp
support_phaser_left_swapkhyp
support_phaser_left_kE
support_phaser_left_swapkE
support_phaser_left_kBA
support_phaser_left_trace
support_phaser_right_khyp
support_phaser_right_swapkhyp
support_phaser_right_kE
support_phaser_right_swapkE
support_phaser_right_kBA
support_phaser_right_trace
support_fuzz_left_khyp
support_fuzz_left_swapkhyp
support_fuzz_left_kE
support_fuzz_left_swapkE
support_fuzz_left_kBA
support_fuzz_left_trace
support_fuzz_right_khyp
support_fuzz_right_swapkhyp
support_fuzz_right_kE
support_fuzz_right_swapkE
support_fuzz_right_kBA
support_fuzz_right_trace
support_spider_left_khyp
support_spider_left_swapkhyp
support_spider_left_kE
support_spider_left_swapkE
su

In [5]:
df

,NEGATED,ALTERNATIVE,RELATION,MEANRATING,SPLIT,support_diag_left_khyp,support_diag_left_swapkhyp,support_diag_left_kE,support_diag_left_swapkE,support_diag_left_kBA,...,subtract_spider_left_kE,subtract_spider_left_swapkE,subtract_spider_left_kBA,subtract_spider_left_trace,subtract_spider_right_khyp,subtract_spider_right_swapkhyp,subtract_spider_right_kE,subtract_spider_right_swapkE,subtract_spider_right_kBA,subtract_spider_right_trace
0,goat,beard,free_association,1.30,train,0.396233,0.045101,0.096631,0.111125,-0.258798,...,0.110688,0.091823,-0.164598,0.020343,0.400928,0.040779,0.167675,0.127458,-0.174756,0.030805
1,alligator,teeth,free_association,1.50,test,1.000000,0.000533,0.241355,0.012153,0.873508,...,0.200841,0.103815,0.131859,0.021550,0.207259,0.043459,0.321007,0.163759,0.142902,0.039610
2,chair,office,hypernym,1.60,train,0.000398,0.177814,0.206201,0.391156,-0.687594,...,0.477197,0.102936,0.625685,0.023763,0.058775,0.026717,0.490305,0.131055,0.658321,0.029549
3,yacht,job,unrelated,1.45,train,1.000000,0.000001,0.085042,0.002347,0.922510,...,0.057536,0.072557,-0.199891,0.021213,1.000000,0.054771,0.159825,0.152999,-0.214037,0.049011
4,goose,ducks,distributional,4.10,test,0.041258,0.067024,0.158364,0.172409,-0.296379,...,0.269108,0.082023,0.153847,0.020267,0.134649,0.206731,0.528213,0.302594,0.219750,0.063217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,trout,living,unrelated,1.00,test,1.000000,0.000029,0.263984,0.006634,0.929129,...,0.110637,0.082846,-0.094445,0.020344,0.258247,0.039289,0.173084,0.121786,-0.099609,0.033790
1227,jacket,ones,unrelated,1.00,train,0.071032,0.244374,0.107561,0.518206,-0.895769,...,0.141472,0.106907,0.090568,0.033182,0.374565,0.034209,0.178482,0.136660,0.094143,0.042291
1228,television,rediffusion,cohyponym,1.40,test,1.000000,0.026171,0.022363,0.113597,-0.680654,...,0.047459,0.023908,0.138348,0.033157,1.000000,0.013581,0.034826,0.035108,0.140179,0.048160
1229,corn,students,unrelated,1.20,test,0.082572,0.006119,0.352505,0.055008,0.687025,...,0.255743,0.079237,0.412639,0.020856,0.036075,0.028366,0.287206,0.113949,0.423243,0.031791


In [6]:
neg_length = 1

df_scores = pd.DataFrame(columns=["swap", "negation", "phaser", "basis", "hyp", "score"])
for col in df.columns[5:]:
    split = col.split("_")
    
    print(split)
    if (len(split) == 6):
        df_scores = df_scores.append(
                {"swap": len(split[4]) == 5, 
                 "negation":split[0], 
                 "phaser":split[0 + neg_length], 
                 "basis":split[1 + neg_length], 
                 "hyp":split[3 + neg_length], 
                 "score":(round(df.corrwith(df[col], method="pearson")["MEANRATING"], 3))}, ignore_index=True)
    else:
        df_scores = df_scores.append(
            {"swap": ("swap" in split[3]), 
             "negation":split[0]
#                  + split[1]
             , 
             "phaser":split[0 + neg_length], 
             "basis":split[1 + neg_length], 
             "hyp":split[2 + neg_length].replace("swap", ""), 
             "score":(round(df.corrwith(df[col], method="pearson")["MEANRATING"], 3))}, ignore_index=True)



['support', 'diag', 'left', 'khyp']
['support', 'diag', 'left', 'swapkhyp']
['support', 'diag', 'left', 'kE']
['support', 'diag', 'left', 'swapkE']
['support', 'diag', 'left', 'kBA']
['support', 'diag', 'left', 'trace']
['support', 'diag', 'right', 'khyp']
['support', 'diag', 'right', 'swapkhyp']
['support', 'diag', 'right', 'kE']
['support', 'diag', 'right', 'swapkE']
['support', 'diag', 'right', 'kBA']
['support', 'diag', 'right', 'trace']
['support', 'phaser', 'left', 'khyp']
['support', 'phaser', 'left', 'swapkhyp']
['support', 'phaser', 'left', 'kE']
['support', 'phaser', 'left', 'swapkE']
['support', 'phaser', 'left', 'kBA']
['support', 'phaser', 'left', 'trace']
['support', 'phaser', 'right', 'khyp']
['support', 'phaser', 'right', 'swapkhyp']
['support', 'phaser', 'right', 'kE']
['support', 'phaser', 'right', 'swapkE']
['support', 'phaser', 'right', 'kBA']
['support', 'phaser', 'right', 'trace']
['support', 'fuzz', 'left', 'khyp']
['support', 'fuzz', 'left', 'swapkhyp']
['suppor

In [7]:
#simple function to sort correlations such that they can easily be copied into excel 
# for table which displays the interaction of different operations        

previous_neg = "none"
print("\multirow{14}*{$CN_{word2}$} ", end = '')
for neg in ['subtract', 'support', "kernel", 'suppker']:  
    if neg != previous_neg:
        if (neg == "idneg"):
            print(" & \multirow{7}*{$\\neg_{sub}$}", end = '')
        else:
#             print("")
            print(neg)
#             print("")
            
            print("\cline{2-9}")
            print(" & \multirow{7}*{$\\neg_{inv}$}", end = '')
    
    
    for conj in ["spider", "phaser", "fuzz", "diag"]:
        for basis in ['right']:
#             print("-----------------------,----------------------")

            if conj == 'diag':
#                 and basis == 'left':
                continue
            
            
            if (previous_neg == neg):
                print("&", end = '')

            print(" & " + conj, end = '')
            if (conj != 'diag'): 
                if basis == 'right':
                    print("$_\\text{c}$", end='')
                else:
                    print("$_\\text{w}$", end='')
                    
            print("& ", end = '')
                
            previous_neg = neg
            for hyp in ["kE", "khyp", "kBA", "trace"]:
                for swap in [False, True]:
                    
        
                    score = df_scores[
                        (df_scores["negation"] == neg) & 
                        (df_scores["phaser"] == conj) & 
                        (df_scores["basis"] == basis) & 
                        (df_scores["hyp"] == hyp) & 
                        (df_scores["swap"] == swap)
                    ]["score"].reset_index()["score"]
                    name = hyp + str(swap) + conj + basis + neg
                    

                    if len(score) == 0:
#                         print(neg, conj, basis, hyp, swap)
                        continue
                    
                    if ("BA" in hyp):
                        score[0] = -1 * score[0]
                    
                        
                    if score[0] > 0.4: 
                        print("\cellcolor{green!45}", end = '')
                    
                    if hyp == "trace":
#                         print(swap, score)
                        print("%.3f \\\\" % round((score[0]), 3))
#                         print(name + " \\\\")
                    else: 
                        print("%.3f & " % round((score[0]), 3), end = '')
#                         print(name + " & ", end = '')
#                         pass

\multirow{14}*{$CN_{word2}$} subtract
\cline{2-9}
 & \multirow{7}*{$\neg_{inv}$} & spider$_\text{c}$& -0.106 & 0.156 & 0.298 & 0.344 & 0.242 & \cellcolor{green!45}0.474 \\
& & phaser$_\text{c}$& -0.265 & -0.343 & 0.301 & -0.260 & 0.265 & -0.317 \\
& & fuzz$_\text{c}$& -0.258 & -0.245 & 0.301 & -0.061 & 0.266 & -0.071 \\
support
\cline{2-9}
 & \multirow{7}*{$\neg_{inv}$} & spider$_\text{c}$& 0.226 & 0.330 & 0.240 & 0.218 & 0.159 & \cellcolor{green!45}0.448 \\
& & phaser$_\text{c}$& -0.077 & 0.013 & 0.235 & 0.082 & 0.150 & 0.274 \\
& & fuzz$_\text{c}$& -0.210 & -0.024 & 0.231 & 0.025 & 0.153 & 0.068 \\
kernel
\cline{2-9}
 & \multirow{7}*{$\neg_{inv}$} & spider$_\text{c}$& -0.120 & 0.088 & 0.237 & 0.254 & 0.145 & \cellcolor{green!45}0.432 \\
& & phaser$_\text{c}$& -0.267 & -0.280 & 0.296 & -0.208 & 0.183 & -0.333 \\
& & fuzz$_\text{c}$& -0.244 & -0.202 & 0.295 & -0.061 & 0.184 & -0.083 \\
suppker
\cline{2-9}
 & \multirow{7}*{$\neg_{inv}$} & spider$_\text{c}$& -0.003 & 0.220 & 0.242 & 0.32

In [8]:
df_scores

,swap,negation,phaser,basis,hyp,score
0,False,support,diag,left,khyp,0.217
1,True,support,diag,left,khyp,0.060
2,False,support,diag,left,kE,-0.262
3,True,support,diag,left,kE,-0.009
4,False,support,diag,left,kBA,-0.131
...,...,...,...,...,...,...
187,True,subtract,spider,right,khyp,0.344
188,False,subtract,spider,right,kE,-0.106
189,True,subtract,spider,right,kE,0.156
190,False,subtract,spider,right,kBA,-0.242
